In [4]:
import time
import os
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()
print(graph.schema)


Node properties are the following:
Person {id: STRING},Nationality {id: STRING},Occupation {id: STRING},Field {id: STRING},Award {id: STRING},Organization {id: STRING},Researcharea {id: STRING},Research area {id: STRING},Field_of_study {id: STRING}
Relationship properties are the following:

The relationships are the following:
(:Person)-[:NATIONALITY]->(:Nationality),(:Person)-[:OCCUPATION]->(:Occupation),(:Person)-[:RESEARCH_FIELD]->(:Field),(:Person)-[:AWARDEE]->(:Award),(:Person)-[:PROFESSOR]->(:Organization),(:Person)-[:AWARD_WINNER]->(:Award),(:Person)-[:AWARD]->(:Award),(:Person)-[:RESEARCH_AREA]->(:Researcharea),(:Person)-[:RESEARCH_AREA]->(:Research area),(:Person)-[:FIELD_OF_STUDY]->(:Field),(:Person)-[:FIELD_OF_STUDY]->(:Field_of_study),(:Person)-[:AWARD_RECEIVED]->(:Award)


In [23]:
start_time = time.time()

# from langchain.chains import GraphCypherQAChain
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4-0125-preview",
    organization=os.getenv('ASCEND_ORG_ID'),
    api_key=os.getenv('SS_OPENAI_API_KEY'),
)
# default_llm = ChatOpenAI(
#     temperature=0,
#     organization=os.getenv('ASCEND_ORG_ID'),
#     api_key=os.getenv('SS_OPENAI_API_KEY'),
# )

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=llm,
    # qa_llm=default_llm, # defulat model "gpt-3.5-turbo"
    qa_llm=llm, # defulat model "gpt-3.5-turbo"
    graph=graph,
    verbose=True,
    return_intermediate_steps=True
)

In [21]:
cypher_chain.invoke("Who is the chemist?")

{'query': 'Who is the chemist?',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': 'cypher\nMATCH (p:Person)-[:OCCUPATION]->(o:Occupation {id: "chemist"})\nRETURN p.id AS Chemist\n'},
  {'context': []}]}

In [24]:
cypher_chain("Who is the chemist?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:OCCUPATION]->(o:Occupation {id: "chemist"})
RETURN p.id AS Chemist

Full Context:
[]

> Finished chain.


{'query': 'Who is the chemist?',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': 'cypher\nMATCH (p:Person)-[:OCCUPATION]->(o:Occupation {id: "chemist"})\nRETURN p.id AS Chemist\n'},
  {'context': []}]}